In [1]:
#Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import numpy as np

In [2]:
#Launch Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Pulling Data from Race Results Page

In [3]:
#Visit Race Results Page 
url = "https://rankings.usatriathlon.org/Event/ViewEvent/301770/1"
browser.visit(url)

In [4]:
#we want the HTML from the page our browser is currently on
html = browser.html


#soupify (parse) the html so we can read it
soup = bs(html, 'html.parser')

In [5]:
#Find the data - in this case where the role = gridcell
athlete_data = soup.find_all(attrs={"role": "gridcell"})
athlete_data

[]

In [6]:
athlete_data[1]

IndexError: list index out of range

In [7]:
#Extract the data and put it into a DataFrame 
count = 0
fullTable = []
currow = []

for cell in athlete_data:
    try:
            text = cell.find("div").text
            currow.append(text)
            count += 1
            if count == 9:
                fullTable.append(currow)
                currow = []
                count = 0
    except Exception as e: 
        print(e)
df = pd.DataFrame(fullTable)

In [8]:
df

""


### Pulling URLS from Main Results Page

In [9]:
#Visit the main page 
url2 = "https://rankings.usatriathlon.org/Event/Events"
browser.visit(url2)

In [10]:
#Grab the HTML and soupify 
html2 = browser.html
soup2= bs(html2, 'html.parser')

In [11]:
#Bring back the URLS and race namesfor each race page 
links = soup2.find_all("a")
link_list = []
race_name = []
for anchor in links:
    links = anchor["href"]
    names = anchor.text
    
    link_list.append(links)
    race_name.append(names)

In [12]:
soup2.find_all("a")[4].text

'Bartow Blarney Triathlon'

In [13]:
#Create a DataFrame to hold the Race Name and the URLS - we'll iterate through this to extract data from each page
race_main = pd.DataFrame()
race_main['Race'] = race_name
race_main['link'] = ['https://rankings.usatriathlon.org'+ link for link in link_list]
#We'll be able to use try/except to filter out the pages that aren't actually race results

In [14]:
race_main

,Race,link
0,\n,https://rankings.usatriathlon.orghttp://usatri...
1,Race Results,https://rankings.usatriathlon.org/Event/Events
2,National Rankings,https://rankings.usatriathlon.org/Rankings/Nat...
3,Athlete Results,https://rankings.usatriathlon.org/RaceResult/A...
4,Bartow Blarney Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
5,College of the Florida Keys Sprint Triathlon/B...,https://rankings.usatriathlon.org/Event/ViewEv...
6,Florida Ultra Week,https://rankings.usatriathlon.org/Event/ViewEv...
7,UCSB Kendra's Race Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
8,Las Olas Fort Lauderdale Triathlon,https://rankings.usatriathlon.org/Event/ViewEv...
9,Pasadena Triathlon 2020,https://rankings.usatriathlon.org/Event/ViewEv...


### Put It All Together

In [15]:
# #check that each page is being visited/reference list
# for index,row in race_main.iterrows(): 
#     url3 = row.link
#     if '/ViewEvent' in url3:
#         race = row.Race
#         try:
#             browser.visit(url3)
#             html3 = browser.html
#             soup3 = bs(html3, 'html.parser')
#             print(race)
#         except Exception as e: 
#             print(e)

In [16]:
browser.quit()

In [17]:
# create an empty df
USAT_data = pd.DataFrame()

#for row in link_data...
for index,row in race_main.iterrows():
    #open a browswer
    browser = Browser('chrome', **executable_path, headless=True)
    #set the url as the link in the dataframe
    url3 = row.link
    #if the url is a race results page (as opposed to a FB link or a URL fragment) then...
    if '/ViewEvent' in url3:
        #set the race variable as the name of the Race
        race = row.Race
        try:
            #visit the browser, grab the html, and soupify it
            browser.visit(url3)
            
            #schleep - there was a mismatch between the time it took the site to load and the time it took to scrape 
            #so this will let splinter chill out for a sec and let us grab what we need
            time.sleep(3)
            
            #resume operations
            html3 = browser.html
            soup3 = bs(html3, 'html.parser')
            
            #find the athlete data
            triathlete_data = soup3.find_all(attrs={"role": "gridcell"})
            
            #get each row of data and store in df 
            count = 0
            fullTable = []
            currow = []
            for cell in triathlete_data:
                #insert another try/except pair for the Florida races, where the first div doesn't have any text
                try:
                    info = cell.find("div").text
                    currow.append(info)
                    count += 1
                    if count == 9:
                        fullTable.append(currow)
                        currow = []
                        count = 0
                except: 
                    next
            df = pd.DataFrame(fullTable)
            
            #add in a column with the race
            df['Race'] = race
            
            #add to the empty df
            USAT_data = USAT_data.append(df)
            
        #if it isn't successful, bring back the name of the race and the error
        except Exception as e: 
            print(race)
            print(e)
            
        #quit browser
        browser.quit()

In [18]:
USAT_data.Race.value_counts()

Huntington's Disease Triathlon Sprint and Olympic - Racing 4 Research To Find a Cure    20
Las Olas Fort Lauderdale Triathlon                                                      20
Tri-owa Indoor Triathlon                                                                20
Tritonman 2020                                                                          20
Iceman Triathlon                                                                        20
Chasing Caterpillars Triathlon 2020                                                     20
Clermont Draft Legal Challenge                                                          20
3rd Annual Tri-Family Racing Lake Pleasant                                              20
Ultraman Florida Triathlon                                                              20
Desert Triathlon                                                                        20
HITS Triathlon Series: Sarasota, FL                                                     20

In [30]:
USAT_data

,Rank,First_Name,Last_Name,Sex,Age,State,Time,Points,Race,Race
0,1,Michael,Alexander,M,53,FL,Age Group,01:02:29.000,87.912,Bartow Blarney Triathlon
1,2,Mark,Hulbert,M,54,FL,Age Group,01:04:06.000,85.695,Bartow Blarney Triathlon
2,3,Brian,Durden,M,35,FL,Age Group,01:05:30.000,83.863,Bartow Blarney Triathlon
3,4,Rodney,Carter,M,42,FL,Age Group,01:05:54.000,83.354,Bartow Blarney Triathlon
4,5,James,Hooppaw,M,27,FL,Age Group,01:06:10.000,83.018,Bartow Blarney Triathlon
...,...,...,...,...,...,...,...,...,...,...
15,16,Jason,Blackman,M,36,FL,Age Group,01:23:43.745,77.732,"HITS Triathlon Series: Sarasota, FL"
16,17,Katie,Hammond,F,35,FL,Age Group,01:24:08.040,85.094,"HITS Triathlon Series: Sarasota, FL"
17,18,Mateus,Arruda,M,33,FL,Age Group,01:24:25.790,77.087,"HITS Triathlon Series: Sarasota, FL"
18,19,Hunter,Carey,M,21,IA,Age Group,01:25:26.741,76.17,"HITS Triathlon Series: Sarasota, FL"


In [31]:
type('Age')

str

In [34]:
del USAT_data['Time']

In [35]:
USAT_data = USAT_data.rename(columns = 
                             {USAT_data.columns[0]:'Rank', 
                              USAT_data.columns[1]:'First_Name',
                              USAT_data.columns[2]:'Last_Name',
                              USAT_data.columns[3]:'Sex',
                              USAT_data.columns[4]:'Age',
                              USAT_data.columns[5]:'State',
                              USAT_data.columns[6]:'Time',
                              USAT_data.columns[7]:'Points', 
                              USAT_data.columns[8]:'Race'})

In [36]:
USAT_data

,Rank,First_Name,Last_Name,Sex,Age,State,Time,Race,Race
0,1,Michael,Alexander,M,53,FL,01:02:29.000,87.912,Bartow Blarney Triathlon
1,2,Mark,Hulbert,M,54,FL,01:04:06.000,85.695,Bartow Blarney Triathlon
2,3,Brian,Durden,M,35,FL,01:05:30.000,83.863,Bartow Blarney Triathlon
3,4,Rodney,Carter,M,42,FL,01:05:54.000,83.354,Bartow Blarney Triathlon
4,5,James,Hooppaw,M,27,FL,01:06:10.000,83.018,Bartow Blarney Triathlon
...,...,...,...,...,...,...,...,...,...
15,16,Jason,Blackman,M,36,FL,01:23:43.745,77.732,"HITS Triathlon Series: Sarasota, FL"
16,17,Katie,Hammond,F,35,FL,01:24:08.040,85.094,"HITS Triathlon Series: Sarasota, FL"
17,18,Mateus,Arruda,M,33,FL,01:24:25.790,77.087,"HITS Triathlon Series: Sarasota, FL"
18,19,Hunter,Carey,M,21,IA,01:25:26.741,76.17,"HITS Triathlon Series: Sarasota, FL"


In [ ]:
USAT_data= USAT_data.set_index('Rank', drop= True)

In [ ]:
USAT_data

In [ ]:
USAT_data.Age= pd.to_numeric(USAT_data.Age)

In [ ]:
type('Age')

In [ ]:
bins = [0,4,9,14,19,24,29,34,39,44,49,54,59,64,69]
labels = ['0-4', '5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69']

In [ ]:
len(bins)

In [ ]:
pd.cut(USAT_data["Age"], bins, labels=labels, include_lowest = True).head()

In [ ]:
USAT_data["Age"] = pd.cut(USAT_data["Age"], bins, labels=labels)
USAT_data.head()

In [ ]:
USAT_data

In [ ]:
USAT_data.to_csv('USAT_Data.csv')